# 🐱🧬 sgRNA Efficiency Prediction with CatBoost

This script evaluates **sgRNA efficiency scores** using **CatBoostRegressor** with **5-fold cross-validation**.  
It includes **feature importance analysis** and **sequence logos** for nucleotide contribution visualization.  

---

## 📦 Requirements
```bash
pip install catboost seaborn logomaker scipy scikit-learn matplotlib pandas numpy


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logomaker
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from scipy.stats import spearmanr, pearsonr

# ============================
# Data Loading and Preprocessing
# ============================

# Read the csv file
# 'HL60' - 'Hct116' - 'HeLa'     "Sniper" - "SpCas" - "xCas"     "WT" - "HF" - "ESP"
df = pd.read_csv("..\\Data\\HL60（n=2076）.csv")

# Extract sequences and labels
sgRNA_seqs = df['sgRNA']
y = df['indel'].values.flatten()

# One-hot encoding for sequences
base_dict = {'A': 0, 'T': 1, 'C': 2, 'G': 3}


def base_to_onehot_flat(seq):
    """Flattened one-hot encoding for compatibility with CatBoost."""
    arr = np.zeros((len(seq) * 4), dtype=int)
    for i, base in enumerate(seq.upper()):
        if base in base_dict:
            arr[i * 4 + base_dict[base]] = 1
    return arr


X = np.array([base_to_onehot_flat(seq) for seq in sgRNA_seqs])

# ============================
# Evaluation Function
# ============================


def evaluate_catboost_5fold(
    X, y,
    iterations=500, depth=10, learning_rate=0.01,
    l2_leaf_reg=3, border_count=64,
    categorical_features=None
):
    """
    Train and evaluate CatBoost on sgRNA data using 5-fold cross-validation.
    Includes feature importance and nucleotide sequence logo.
    """
    kf = KFold(n_splits=5, shuffle=True, random_state=98)
    spearman_scores = []
    pearson_scores = []

    all_feature_importances = np.zeros(X.shape[1])
    seq_length = X.shape[1] // 4
    all_nucleotide_importances = np.zeros((seq_length, 4))

    for fold, (train_idx, test_idx) in enumerate(kf.split(X), 1):
        print(f"\n--- Fold {fold} ---")
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Define and train CatBoost regressor
        model = CatBoostRegressor(
            iterations=iterations,
            depth=depth,
            learning_rate=learning_rate,
            l2_leaf_reg=l2_leaf_reg,
            border_count=border_count,
            loss_function='RMSE',
            cat_features=categorical_features,
            verbose=False
        )
        model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=0)

        # Feature importances
        feature_importances = model.get_feature_importance()
        all_feature_importances += feature_importances

        # Nucleotide importances per position
        for pos in range(seq_length):
            pos_importances = feature_importances[pos * 4:(pos + 1) * 4]
            all_nucleotide_importances[pos] += pos_importances

        # Approximate number of parameters
        num_params = model.tree_count_ * depth * 2
        print(f"Number of Parameters (estimated): {num_params}")

        # Predictions
        y_pred = model.predict(X_test)

        # Correlations
        spearman_corr, _ = spearmanr(y_test, y_pred)
        pearson_corr, _ = pearsonr(y_test, y_pred)
        spearman_scores.append(spearman_corr)
        pearson_scores.append(pearson_corr)

        print(f"Spearman: {spearman_corr:.4f}, Pearson: {pearson_corr:.4f}")

    # ============================
    # Results Summary
    # ============================

    print("\n=== Average Results over 5 folds ===")
    print(
        f"Avg Spearman: {np.mean(spearman_scores):.4f} ± {np.std(spearman_scores):.4f}")
    print(
        f"Avg Pearson : {np.mean(pearson_scores):.4f} ± {np.std(pearson_scores):.4f}")

    mean_feature_importances = all_feature_importances / 5
    mean_nucleotide_importances = all_nucleotide_importances / 5

    # ============================
    # Plots
    # ============================

    # Bar plot of feature importances
    plt.figure(figsize=(10, 6))
    plt.bar(range(X.shape[1]), mean_feature_importances)
    plt.xlabel('Feature Index')
    plt.ylabel('Mean Importance')
    plt.title('Feature Importances (Mean over 5 Folds)')
    plt.show()

    # Sequence logo for nucleotide importances
    df_importances = pd.DataFrame(
        mean_nucleotide_importances, columns=["A", "T", "C", "G"])
    logo = logomaker.Logo(df_importances)
    logo.style_spines(visible=False)
    logo.style_xticks(rotation=90)
    logo.ax.set_xticks(np.arange(seq_length))
    logo.ax.set_xticklabels(np.arange(1, seq_length + 1))
    logo.ax.set_title("Nucleotide Importances - Sequence Logo")
    plt.show()
    
print("\n### Default Parameters Evaluation ###")
evaluate_catboost_5fold(X, y)

# sgRNA Efficiency Prediction with CatBoost + Optuna

This code  demonstrates how to optimize a **CatBoostRegressor** for predicting sgRNA efficiency scores using **Optuna**.  
The optimization metric is a weighted combination of **Spearman** and **Pearson correlations**.

---

## 📦 Requirements
```bash
pip install catboost optuna scipy scikit-learn matplotlib


In [ ]:
# --- Imports ---
import optuna
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt

# --- Train/test split (80/20) ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=98
)

# --- Optuna objective function ---
def objective(trial):
    # Suggest hyperparameters
    iterations = trial.suggest_int('iterations', 500, 2000)  # higher floor, more stable
    depth = trial.suggest_int('depth', 4, 10)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 0.1)
    l2_leaf_reg = trial.suggest_loguniform('l2_leaf_reg', 1e-2, 10)
    border_count = trial.suggest_categorical('border_count', [32, 64])

    # Define CatBoost model
    model = CatBoostRegressor(
        iterations=iterations,
        depth=depth,
        learning_rate=learning_rate,
        l2_leaf_reg=l2_leaf_reg,
        border_count=border_count,
        loss_function='RMSE',
        verbose=False,
        random_seed=98
    )

    # Train with early stopping
    model.fit(
        X_train, y_train,
        eval_set=(X_test, y_test),
        early_stopping_rounds=80,
        verbose=False
    )

    # Predictions
    y_pred = model.predict(X_test)

    # Evaluate with Spearman + Pearson
    spearman_corr, _ = spearmanr(y_test, y_pred)
    pearson_corr, _ = pearsonr(y_test, y_pred)

    # Weighted objective (tune weights if you want)
    return 0.7 * spearman_corr + 0.3 * pearson_corr

# --- Run Optuna study ---
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# --- Show best parameters ---
print("Best hyperparameters:", study.best_params)
print("Best objective value:", study.best_value)

# --- Retrain best model on train data ---
best_params = study.best_params
final_model = CatBoostRegressor(
    iterations=best_params['iterations'],
    depth=best_params['depth'],
    learning_rate=best_params['learning_rate'],
    l2_leaf_reg=best_params['l2_leaf_reg'],
    border_count=best_params['border_count'],
    loss_function='RMSE',
    verbose=False,
    random_seed=98
)
final_model.fit(X_train, y_train, early_stopping_rounds=50, verbose=False)

# --- Evaluate final model ---
y_pred_test = final_model.predict(X_test)
spearman_corr_test, _ = spearmanr(y_test, y_pred_test)
pearson_corr_test, _ = pearsonr(y_test, y_pred_test)

print(f"Test Set Spearman: {spearman_corr_test:.4f}")
print(f"Test Set Pearson: {pearson_corr_test:.4f}")

# --- Optional: visualize study history ---
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_param_importances(study)


In [ ]:
if __name__ == "__main__":
    print("\n### Default Parameters Evaluation ###")
    evaluate_catboost_5fold(X, y)

    print("\n### Best Parameters Evaluation ###")
    evaluate_catboost_5fold(
        X, y,
        iterations=best_params['iterations'],
        depth=best_params['depth'],
        learning_rate=best_params['learning_rate'],
        l2_leaf_reg=best_params['l2_leaf_reg'],
        border_count=best_params['border_count']
    )